# Monitoramento do desempenho do Banco de Dados SQL do Microsoft Azure e da Instância Gerenciada de SQL usando DMV

O Banco de Dados SQL do Microsoft Azure e a Instância Gerenciada de SQL do Azure permitem um subconjunto de modos de exibições de gerenciamento dinâmico para diagnosticar problemas de desempenho, que podem ser causados por consultas bloqueadas ou de execução longa, gargalos de recursos, planos de consulta insatisfatórios e assim por diante. Este artigo fornece informações sobre como detectar problemas de desempenho comuns usando exibições de gerenciamento dinâmico.

O Banco de Dados SQL do Microsoft Azure e a Instância Gerenciada de SQL do Azure dão suporte parcial a três categorias de exibições de gerenciamento dinâmico:

- Exibições de gerenciamento dinâmico relacionadas ao banco de dados.
- Exibições de gerenciamento dinâmico relacionadas à execução.
- Exibições de gerenciamento dinâmico relacionadas à transação.

Para obter informações detalhadas sobre exibições de gerenciamento dinâmico, consulte [exibições e funções de gerenciamento dinâmico (Transact-SQL)](https://docs.microsoft.com/pt-br/sql/relational-databases/system-dynamic-management-views/system-dynamic-management-views).

## Monitorar com insights de SQL

O [Azure Monitor insights do SQL](https://docs.microsoft.com/pt-br/azure/azure-monitor/insights/sql-insights-overview) é uma ferramenta para monitorar Instâncias Gerenciadas de SQL do Azure, Banco de Dados SQL do Azure e Instâncias do SQL Server em VMs do SQL do Azure. Esse serviço usa um agente remoto para capturar dados de DMVs (exibições de gerenciamento dinâmico) e roteia os dados para o Azure Log Analytics, onde podem ser monitorados e analisados. Você pode exibir esses dados do [Azure Monitor](https://docs.microsoft.com/pt-br/azure/azure-monitor/overview) nos modos de exibição fornecidos ou acessar os dados de log diretamente para executar consultas e analisar tendências. Para começar a usar o Azure Monitor insights do SQL, consulte [habilitar insights do SQL](https://docs.microsoft.com/pt-br/azure/azure-monitor/insights/sql-insights-enable).

## Permissões

No Banco de Dados SQL do Azure, consultar uma exibição de gerenciamento dinâmico requer permissões VIEW DATABASE STATE. A permissão VIEW DATABASE STATE retorna informações sobre todos os objetos no banco de dados atual. Para conceder a permissão VIEW DATABASE STATE a um usuário específico do banco de dados, execute a seguinte consulta:

In [ ]:
GRANT VIEW DATABASE STATE TO database_user;

No Instância Gerenciada de SQL do Azure, consultar uma exibição de gerenciamento dinâmico requer permissões VIEW SERVER STATE. Para saber mais, veja [exibições de gerenciamento dinâmico do sistema](https://docs.microsoft.com/pt-br/sql/relational-databases/system-dynamic-management-views/system-dynamic-management-views#required-permissions).

Em uma instância do SQL Server e na Instância Gerenciada de SQL do Azure, as exibições de gerenciamento dinâmico retornam informações de estado do servidor. Em um Banco de Dados SQL do Azure, elas retornam informações relacionadas apenas ao seu banco de dados lógico atual.

Este artigo contém uma coleção de consultas DMV que você pode executar usando SQL Server Management Studio ou Azure Data Studio para detectar os seguintes tipos de problemas de desempenho de consulta:

- [Identificando consultas relacionadas ao consumo excessivo de CPU](https://docs.microsoft.com/pt-br/azure/azure-sql/database/monitoring-with-dmvs#identify-cpu-performance-issues)
- [PAGELATCH\_ \* e WRITE\_LOG esperas relacionadas aos gargalos de e/s](https://docs.microsoft.com/pt-br/azure/azure-sql/database/monitoring-with-dmvs#identify-io-performance-issues)
- [PAGELATCH\_ \* esperas causaram contenção de bytTempDB](https://docs.microsoft.com/pt-br/azure/azure-sql/database/monitoring-with-dmvs#identify-tempdb-performance-issues)
- [RESOURCE\_SEMAHPORE esperas causadas por problemas de espera de concessão de memória](https://docs.microsoft.com/pt-br/azure/azure-sql/database/monitoring-with-dmvs#identify-memory-grant-wait-performance-issues)
- [Identificando tamanhos de banco de dados e objeto](https://docs.microsoft.com/pt-br/azure/azure-sql/database/monitoring-with-dmvs#calculating-database-and-objects-sizes)
- [Recuperando informações sobre sessões ativas](https://docs.microsoft.com/pt-br/azure/azure-sql/database/monitoring-with-dmvs#monitoring-connections)
- [Recuperar informações de uso de recursos de todo o sistema e de banco de dados](https://docs.microsoft.com/pt-br/azure/azure-sql/database/monitoring-with-dmvs#monitor-resource-use)
- [Recuperando informações de desempenho de consulta](https://docs.microsoft.com/pt-br/azure/azure-sql/database/monitoring-with-dmvs#monitoring-query-performance)

## Identificar problemas de desempenho da CPU

Se o consumo de CPU estiver acima de 80% por longos períodos de tempo, considere as seguintes etapas de solução de problemas:

### [](https://docs.microsoft.com/pt-br/azure/azure-sql/database/monitoring-with-dmvs#the-cpu-issue-is-occurring-now)O problema de CPU está ocorrendo agora

Se o problema está ocorrendo no momento, há dois cenários possíveis:

#### [](https://docs.microsoft.com/pt-br/azure/azure-sql/database/monitoring-with-dmvs#many-individual-queries-that-cumulatively-consume-high-cpu)Muitas consultas individuais que consomem cumulativamente alta utilização da CPU

Use a consulta a seguir para identificar os hashes de consulta top:

In [1]:
PRINT '-- top 10 Active CPU Consuming Queries (aggregated)--';
SELECT TOP 10 GETDATE() runtime, *
FROM (SELECT query_stats.query_hash, SUM(query_stats.cpu_time) 'Total_Request_Cpu_Time_Ms', SUM(logical_reads) 'Total_Request_Logical_Reads', MIN(start_time) 'Earliest_Request_start_Time', COUNT(*) 'Number_Of_Requests', SUBSTRING(REPLACE(REPLACE(MIN(query_stats.statement_text), CHAR(10), ' '), CHAR(13), ' '), 1, 256) AS "Statement_Text"
    FROM (SELECT req.*, SUBSTRING(ST.text, (req.statement_start_offset / 2)+1, ((CASE statement_end_offset WHEN -1 THEN DATALENGTH(ST.text)ELSE req.statement_end_offset END-req.statement_start_offset)/ 2)+1) AS statement_text
          FROM sys.dm_exec_requests AS req
                CROSS APPLY sys.dm_exec_sql_text(req.sql_handle) AS ST ) AS query_stats
    GROUP BY query_hash) AS t
ORDER BY Total_Request_Cpu_Time_Ms DESC;

-- top 10 Active CPU Consuming Queries (aggregated)--

(7 rows affected)

Total execution time: 00:00:09.230

runtime,query_hash,Total_Request_Cpu_Time_Ms,Total_Request_Logical_Reads,Earliest_Request_start_Time,Number_Of_Requests,Statement_Text
2021-11-05 12:37:29.900,0x8483E126EE5CA053,1166,5,2021-11-05 11:38:47.243,1,"SELECT * FROM RS_PESSOAS_PERFIS_LOG WHERE CODIGOPESSOA IN (2178032, 2742478, 2707351) ORDER BY DATAENVIO DESC"
2021-11-05 12:37:29.900,0x809C532871BDBB4B,146,658,2021-11-05 12:34:34.930,1,"SELECT Status_Matricula , SUM(Qtd_Matriculas) AS Qtd_Matriculas , SUM(Qtd_Matriculas_Ajuste) AS Qtd_Matriculas_Ajuste , SUM(Qtd_Matriculas_Iniciadas) AS Qtd_Matriculas_Iniciadas , SUM(Qtd_Matriculas + Qtd_Matriculas_Ajuste) AS total into #tm"
2021-11-05 12:37:29.900,0x9B31741EA6739328,30,22,2021-11-05 12:37:19.947,1,"SELECT dtb.name AS [Name], dtb.database_id AS [ID], CAST(has_dbaccess(dtb.name) AS bit) AS [IsAccessible] FROM master.sys.databases AS dtb ORDER BY [Name] ASC"
2021-11-05 12:37:29.900,0x500A9CB8368B1588,17,695,2021-11-05 12:36:41.153,1,"SELECT top 200 TURMADATAINICIO, TURMADATATERMINO, UNIDADE, CURSO, ALUNODATAMATRICULA, HISTORICO FROM FCO_NOP_002A where ALUNOCPF = '701.635.887-91'"
2021-11-05 12:37:29.900,0x270002CDB8A02040,13,416,2021-11-05 12:37:20.737,1,"SELECT TOP 10 GETDATE() runtime, * FROM (SELECT query_stats.query_hash, SUM(query_stats.cpu_time) 'Total_Request_Cpu_Time_Ms', SUM(logical_reads) 'Total_Request_Logical_Reads', MIN(start_time) 'Earliest_Request_start_Time', COUNT(*) 'Number_Of_Requests',"
2021-11-05 12:37:29.900,0x945DE7387E25D067,7,118,2021-11-05 12:37:27.153,1,"WITH last_hist_rec AS ( SELECT ROW_NUMBER() OVER (PARTITION BY job_id ORDER BY run_date DESC, run_time DESC) AS [RowNum] , job_id , run_date AS [last_run_date] , run_time AS [las"
2021-11-05 12:37:29.900,NULL,0,24,2021-10-15 16:30:58.350,1,sp_server_diagnostics


#### Consultas de longa execução que consomem CPU ainda estão em execução

Use a consulta a seguir para identificar os hashes de consulta:

In [3]:
PRINT '--top 10 Active CPU Consuming Queries by sessions--';
SELECT
    TOP 10
    req.session_id
    ,req.start_time
    ,cpu_time 'cpu_time_ms'
    ,OBJECT_NAME(ST.objectid, ST.dbid) 'ObjectName'
    ,SUBSTRING(REPLACE(REPLACE(SUBSTRING(ST.text, (req.statement_start_offset / 2)+1, ((CASE statement_end_offset WHEN -1 THEN DATALENGTH(ST.text)ELSE req.statement_end_offset END-req.statement_start_offset)/ 2)+1), CHAR(10), ' '), CHAR(13), ' '), 1, 512) AS statement_text
FROM
    sys.dm_exec_requests AS req
    CROSS APPLY sys.dm_exec_sql_text(req.sql_handle) AS ST
ORDER BY cpu_time DESC;
GO

--top 10 Active CPU Consuming Queries by sessions--

(10 rows affected)

Total execution time: 00:00:11.182

session_id,start_time,cpu_time_ms,ObjectName,statement_text
52,2021-11-05 11:38:47.243,1166,NULL,"SELECT * FROM RS_PESSOAS_PERFIS_LOG WHERE CODIGOPESSOA IN (2178032, 2742478, 2707351) ORDER BY DATAENVIO DESC"
89,2021-11-05 12:34:34.930,146,NULL,"SELECT Status_Matricula , SUM(Qtd_Matriculas) AS Qtd_Matriculas , SUM(Qtd_Matriculas_Ajuste) AS Qtd_Matriculas_Ajuste , SUM(Qtd_Matriculas_Iniciadas) AS Qtd_Matriculas_Iniciadas , SUM(Qtd_Matriculas + Qtd_Matriculas_Ajuste) AS total into #tmpcalculos FROM CAD_PRODUCAO_CODEP_ANALITICO x WHERE Ano_Producao = 2021 AND Mes_Producao = 10 and Codigolote <> 658 and not EXISTS( SELECT 1 FROM CAD_PRODUCAO_CODEP_ANALITICO cp WHERE cp.Ano_Producao = 2021 AND Mes_"
82,2021-11-05 12:39:39.010,10,NULL,"SELECT TOP 10 req.session_id ,req.start_time ,cpu_time 'cpu_time_ms' ,OBJECT_NAME(ST.objectid, ST.dbid) 'ObjectName' ,SUBSTRING(REPLACE(REPLACE(SUBSTRING(ST.text, (req.statement_start_offset / 2)+1, ((CASE statement_end_offset WHEN -1 THEN DATALENGTH(ST.text)ELSE req.statement_end_offset END-req.statement_start_offset)/ 2)+1), CHAR(10), ' '), CHAR(13), ' '), 1, 512) AS statement_text FROM sys.dm_exec_requests AS req CROSS APPLY sys.dm_exec_sql_text(req.sql_handle) AS"
60,2021-11-05 12:38:38.113,5,NULL,"SELECT top 200 TURMADATAINICIO, TURMADATATERMINO, UNIDADE, CURSO, ALUNODATAMATRICULA, HISTORICO FROM FCO_NOP_002A where ALUNOCPF = ''"
83,2021-11-05 12:39:01.337,4,NULL,"SELECT top 200 TURMADATAINICIO, TURMADATATERMINO, UNIDADE, CURSO, ALUNODATAMATRICULA, HISTORICO FROM FCO_NOP_002A where ALUNOCPF = '174.418.667-74'"
84,2021-11-05 12:39:02.257,4,NULL,"SELECT top 200 TURMADATAINICIO, TURMADATATERMINO, UNIDADE, CURSO, ALUNODATAMATRICULA, HISTORICO FROM FCO_NOP_002A where ALUNOCPF = '530.893.748-62'"
65,2021-11-05 12:38:49.190,4,NULL,"SELECT top 200 TURMADATAINICIO, TURMADATATERMINO, UNIDADE, CURSO, ALUNODATAMATRICULA, HISTORICO FROM FCO_NOP_002A where ALUNOCPF = ''"
91,2021-11-05 12:39:03.340,4,NULL,"SELECT top 200 TURMADATAINICIO, TURMADATATERMINO, UNIDADE, CURSO, ALUNODATAMATRICULA, HISTORICO FROM FCO_NOP_002A where ALUNOCPF = '280.816.898-50'"
92,2021-11-05 12:39:04.277,4,NULL,"SELECT top 200 TURMADATAINICIO, TURMADATATERMINO, UNIDADE, CURSO, ALUNODATAMATRICULA, HISTORICO FROM FCO_NOP_002A where ALUNOCPF = '047.943.750-50'"
93,2021-11-05 12:39:07.930,4,NULL,"SELECT top 200 TURMADATAINICIO, TURMADATATERMINO, UNIDADE, CURSO, ALUNODATAMATRICULA, HISTORICO FROM FCO_NOP_002A where ALUNOCPF = '089.371.447-00'"


### O problema da CPU ocorreu no passado

Se o problema ocorreu no passado e você deseja analisar a causa raiz, use o[Repositório de Consultas](https://docs.microsoft.com/pt-br/sql/relational-databases/performance/monitoring-performance-by-using-the-query-store). Os usuários com acesso de banco de dados podem usar o T-SQL para consultar os dados dos Repositório de Consultas. As configurações padrão dos Repositório de Dados de Consultas usam uma granularidade de 1 hora. Use a consulta a seguir para examinar a atividade para consultas de consumo de CPU alta. Essa consulta retorna as consultas de consumo de CPU 15 principais. Lembre-se de alterar `rsi.start_time >= DATEADD(hour, -2, GETUTCDATE()`:

In [1]:
-- Top 15 CPU consuming queries by query hash
-- note that a query  hash can have many query id if not parameterized or not parameterized properly
-- it grabs a sample query text by min
WITH AggregatedCPU AS (SELECT q.query_hash, SUM(count_executions * avg_cpu_time / 1000.0) AS total_cpu_millisec, SUM(count_executions * avg_cpu_time / 1000.0)/ SUM(count_executions) AS avg_cpu_millisec, MAX(rs.max_cpu_time / 1000.00) AS max_cpu_millisec, MAX(max_logical_io_reads) max_logical_reads, COUNT(DISTINCT p.plan_id) AS number_of_distinct_plans, COUNT(DISTINCT p.query_id) AS number_of_distinct_query_ids, SUM(CASE WHEN rs.execution_type_desc='Aborted' THEN count_executions ELSE 0 END) AS Aborted_Execution_Count, SUM(CASE WHEN rs.execution_type_desc='Regular' THEN count_executions ELSE 0 END) AS Regular_Execution_Count, SUM(CASE WHEN rs.execution_type_desc='Exception' THEN count_executions ELSE 0 END) AS Exception_Execution_Count, SUM(count_executions) AS total_executions, MIN(qt.query_sql_text) AS sampled_query_text
                       FROM sys.query_store_query_text AS qt
                            JOIN sys.query_store_query AS q ON qt.query_text_id=q.query_text_id
                            JOIN sys.query_store_plan AS p ON q.query_id=p.query_id
                            JOIN sys.query_store_runtime_stats AS rs ON rs.plan_id=p.plan_id
                            JOIN sys.query_store_runtime_stats_interval AS rsi ON rsi.runtime_stats_interval_id=rs.runtime_stats_interval_id
                       WHERE rs.execution_type_desc IN ('Regular', 'Aborted', 'Exception')AND rsi.start_time>=DATEADD(HOUR, -2, GETUTCDATE())
                       GROUP BY q.query_hash), OrderedCPU AS (SELECT query_hash, total_cpu_millisec, avg_cpu_millisec, max_cpu_millisec, max_logical_reads, number_of_distinct_plans, number_of_distinct_query_ids, total_executions, Aborted_Execution_Count, Regular_Execution_Count, Exception_Execution_Count, sampled_query_text, ROW_NUMBER() OVER (ORDER BY total_cpu_millisec DESC, query_hash ASC) AS RN
                                                              FROM AggregatedCPU)
SELECT OD.query_hash, OD.total_cpu_millisec, OD.avg_cpu_millisec, OD.max_cpu_millisec, OD.max_logical_reads, OD.number_of_distinct_plans, OD.number_of_distinct_query_ids, OD.total_executions, OD.Aborted_Execution_Count, OD.Regular_Execution_Count, OD.Exception_Execution_Count, OD.sampled_query_text, OD.RN
FROM OrderedCPU AS OD
WHERE OD.RN<=15
ORDER BY total_cpu_millisec DESC;

: Msg 208, Level 16, State 1, Line 4
Invalid object name 'sys.query_store_query_text'.

Total execution time: 00:00:00.038

<span style=" font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">Depois de identificar as consultas com problemas, é hora de ajustar as consultas para reduzir a utilização da CPU. Se você não tiver tempo para ajustar as consultas, você também pode optar por atualizar o SLO do banco de dados para contornar o problema.</span>

## Identificar problemas de desempenho de e/s

Ao identificar problemas de desempenho de e/s, os tipos de espera superior associados a problemas de e/s são:

- `PAGEIOLATCH_*`
    
    Para problemas de e/s de arquivo de dados (incluindo `PAGEIOLATCH_SH`, `PAGEIOLATCH_EX`, `PAGEIOLATCH_UP`). Se tiver o nome do tipo de espera e/s nele, ele aponta para um problema de e/s. Se não houver nenhuma e/s o nome de espera de trava de página, ela aponta para um tipo diferente de problema (por exemplo, a contenção de tempdb).
    
- `WRITE_LOG`
    
    Para problemas de e/s de log de transações.
    

### [](https://docs.microsoft.com/pt-br/azure/azure-sql/database/monitoring-with-dmvs#if-the-io-issue-is-occurring-right-now)Se o problema de e/s estiver ocorrendo no momento

Use o [. DM exec\_requests](https://docs.microsoft.com/pt-br/sql/relational-databases/system-dynamic-management-views/sys-dm-exec-requests-transact-sql) ou [DM os\_waiting\_tasks](https://docs.microsoft.com/pt-br/sql/relational-databases/system-dynamic-management-views/sys-dm-os-waiting-tasks-transact-sql) para ver os `wait_type` e `wait_time`.

#### Identificar os dados e registrar o uso de e/s

Use a seguinte consulta para identificar os dados e uso de e/s de log. Se a e/s de dados ou de log estiver acima de 80%, isso significa que os usuários usaram a e/s disponível para a camada de serviço do Banco de Dados SQL.

In [ ]:
SELECT end_time, avg_data_io_percent, avg_log_write_percent
FROM sys.dm_db_resource_stats
ORDER BY end_time DESC;

Se tiver sido atingido o limite de e/s, você tem duas opções:

- Opção 1: Atualizar o tamanho da computação ou camada de serviço
- Opção 2: Identificar e ajustar as consultas que consomem a maioria das e/s.

#### Visualizar e/s relacionada ao buffer usando o Repositório de Consultas

Para a opção 2, você pode usar a consulta a seguir em relação ao Repositório de Dados de Consultas para e/s relacionada ao buffer para exibir as últimas duas horas de atividade controlada:

In [ ]:
-- top queries that waited on buffer
-- note these are finished queries
WITH Aggregated AS (SELECT q.query_hash, SUM(total_query_wait_time_ms) total_wait_time_ms, SUM(total_query_wait_time_ms / avg_query_wait_time_ms) AS total_executions, MIN(qt.query_sql_text) AS sampled_query_text, MIN(wait_category_desc) AS wait_category_desc
                    FROM sys.query_store_query_text AS qt
                         JOIN sys.query_store_query AS q ON qt.query_text_id=q.query_text_id
                         JOIN sys.query_store_plan AS p ON q.query_id=p.query_id
                         JOIN sys.query_store_wait_stats AS waits ON waits.plan_id=p.plan_id
                         JOIN sys.query_store_runtime_stats_interval AS rsi ON rsi.runtime_stats_interval_id=waits.runtime_stats_interval_id
                    WHERE wait_category_desc='Buffer IO' AND rsi.start_time>=DATEADD(HOUR, -2, GETUTCDATE())
                    GROUP BY q.query_hash), Ordered AS (SELECT query_hash, total_executions, total_wait_time_ms, sampled_query_text, wait_category_desc, ROW_NUMBER() OVER (ORDER BY total_wait_time_ms DESC, query_hash ASC) AS RN
                                                        FROM Aggregated)
SELECT OD.query_hash, OD.total_executions, OD.total_wait_time_ms, OD.sampled_query_text, OD.wait_category_desc, OD.RN
FROM Ordered AS OD
WHERE OD.RN<=15
ORDER BY total_wait_time_ms DESC;
GO

#### Exibir o log total e/s para WRITELOG espera

Se o tipo de espera for `WRITELOG`, use a seguinte consulta para exibir e/s de log total pela instrução:

In [ ]:
-- Top transaction log consumers
-- Adjust the time window by changing
-- rsi.start_time >= DATEADD(hour, -2, GETUTCDATE())
WITH AggregatedLogUsed
AS (SELECT q.query_hash,
           SUM(count_executions * avg_cpu_time / 1000.0) AS total_cpu_millisec,
           SUM(count_executions * avg_cpu_time / 1000.0) / SUM(count_executions) AS avg_cpu_millisec,
           SUM(count_executions * avg_log_bytes_used) AS total_log_bytes_used,
           MAX(rs.max_cpu_time / 1000.00) AS max_cpu_millisec,
           MAX(max_logical_io_reads) max_logical_reads,
           COUNT(DISTINCT p.plan_id) AS number_of_distinct_plans,
           COUNT(DISTINCT p.query_id) AS number_of_distinct_query_ids,
           SUM(   CASE
                      WHEN rs.execution_type_desc = 'Aborted' THEN
                          count_executions
                      ELSE
                          0
                  END
              ) AS Aborted_Execution_Count,
           SUM(   CASE
                      WHEN rs.execution_type_desc = 'Regular' THEN
                          count_executions
                      ELSE
                          0
                  END
              ) AS Regular_Execution_Count,
           SUM(   CASE
                      WHEN rs.execution_type_desc = 'Exception' THEN
                          count_executions
                      ELSE
                          0
                  END
              ) AS Exception_Execution_Count,
           SUM(count_executions) AS total_executions,
           MIN(qt.query_sql_text) AS sampled_query_text
    FROM sys.query_store_query_text AS qt
        JOIN sys.query_store_query AS q
            ON qt.query_text_id = q.query_text_id
        JOIN sys.query_store_plan AS p
            ON q.query_id = p.query_id
        JOIN sys.query_store_runtime_stats AS rs
            ON rs.plan_id = p.plan_id
        JOIN sys.query_store_runtime_stats_interval AS rsi
            ON rsi.runtime_stats_interval_id = rs.runtime_stats_interval_id
    WHERE rs.execution_type_desc IN ( 'Regular', 'Aborted', 'Exception' )
          AND rsi.start_time >= DATEADD(HOUR, -2, GETUTCDATE())
    GROUP BY q.query_hash),
     OrderedLogUsed
AS (SELECT query_hash,
           total_log_bytes_used,
           number_of_distinct_plans,
           number_of_distinct_query_ids,
           total_executions,
           Aborted_Execution_Count,
           Regular_Execution_Count,
           Exception_Execution_Count,
           sampled_query_text,
           ROW_NUMBER() OVER (ORDER BY total_log_bytes_used DESC, query_hash ASC) AS RN
    FROM AggregatedLogUsed)
SELECT OD.total_log_bytes_used,
       OD.number_of_distinct_plans,
       OD.number_of_distinct_query_ids,
       OD.total_executions,
       OD.Aborted_Execution_Count,
       OD.Regular_Execution_Count,
       OD.Exception_Execution_Count,
       OD.sampled_query_text,
       OD.RN
FROM OrderedLogUsed AS OD
WHERE OD.RN <= 15
ORDER BY total_log_bytes_used DESC;
GO

## Identificar `tempdb` problemas de desempenho

Ao identificar problemas de desempenho de e/s, os principais tipos de espera de associado `tempdb` é de problemas `PAGELATCH_*` (não `PAGEIOLATCH_*`). No entanto, `PAGELATCH_*` esperas sempre significam que você tem `tempdb` contenção. Essa espera também pode significar que você tenha a contenção de página de dados de objeto de usuário devido a solicitações simultâneas, visando a mesma página de dados. Para confirmar ainda mais a contenção de`tempdb`, use [sys.dm\_exec\_requestss](https://docs.microsoft.com/pt-br/sql/relational-databases/system-dynamic-management-views/sys-dm-exec-requests-transact-sql) para confirmar que o valor de wait\_resource começa com `2:x:y` onde 2 é `tempdb` a ID do banco de dados, `x` é a ID do arquivo e `y` é a ID de página.

Para a contenção de tempdb, um método comum é reduzir ou reescrever o código do aplicativo que se baseia em `tempdb`. Áreas de uso comum `tempdb` incluem:

- Tabelas temporárias
- Variáveis de tabela
- Parâmetros com valor de tabela
- Uso do repositório de versão ( associado com transações de execução prolongada)
- Consultas que têm planos de consulta que usam classificações e junções de hash spools

### Principais consultas que usam variáveis de tabela e tabelas temporárias

Use a consulta a seguir para identificar as principais consultas que usam tabelas temporárias e variáveis de tabela:

In [ ]:
SELECT plan_handle, execution_count, query_plan
INTO #tmpPlan
FROM sys.dm_exec_query_stats
     CROSS APPLY sys.dm_exec_query_plan(plan_handle);
GO

WITH XMLNAMESPACES('http://schemas.microsoft.com/sqlserver/2004/07/showplan' AS sp)
SELECT plan_handle, stmt.stmt_details.value('@Database', 'varchar(max)') 'Database', stmt.stmt_details.value('@Schema', 'varchar(max)') 'Schema', stmt.stmt_details.value('@Table', 'varchar(max)') 'table'
INTO #tmp2
FROM(SELECT CAST(query_plan AS XML) sqlplan, plan_handle FROM #tmpPlan) AS p
    CROSS APPLY sqlplan.nodes('//sp:Object') AS stmt(stmt_details);
GO

SELECT t.plan_handle, [Database], [Schema], [table], execution_count
FROM(SELECT DISTINCT plan_handle, [Database], [Schema], [table]
     FROM #tmp2
     WHERE [table] LIKE '%@%' OR [table] LIKE '%#%') AS t
    JOIN #tmpPlan AS t2 ON t.plan_handle=t2.plan_handle;

### Identificar as transações de longa execução

Use a consulta a seguir para identificar transações de execução longas. Transações de longa execução impedem a limpeza do repositório de versão.

In [ ]:
SELECT DB_NAME(dtr.database_id) 'database_name',
       sess.session_id,
       atr.name AS 'tran_name',
       atr.transaction_id,
       transaction_type,
       transaction_begin_time,
       database_transaction_begin_time transaction_state,
       is_user_transaction,
       sess.open_transaction_count,
       LTRIM(RTRIM(REPLACE(
                              REPLACE(
                                         SUBSTRING(
                                                      SUBSTRING(
                                                                   txt.text,
                                                                   (req.statement_start_offset / 2) + 1,
                                                                   ((CASE req.statement_end_offset
                                                                         WHEN -1 THEN
                                                                             DATALENGTH(txt.text)
                                                                         ELSE
                                                                             req.statement_end_offset
                                                                     END - req.statement_start_offset
                                                                    ) / 2
                                                                   ) + 1
                                                               ),
                                                      1,
                                                      1000
                                                  ),
                                         CHAR(10),
                                         ' '
                                     ),
                              CHAR(13),
                              ' '
                          )
                  )
            ) Running_stmt_text,
       recenttxt.text 'MostRecentSQLText'
FROM sys.dm_tran_active_transactions AS atr
    INNER JOIN sys.dm_tran_database_transactions AS dtr
        ON dtr.transaction_id = atr.transaction_id
    LEFT JOIN sys.dm_tran_session_transactions AS sess
        ON sess.transaction_id = atr.transaction_id
    LEFT JOIN sys.dm_exec_requests AS req
        ON req.session_id = sess.session_id
           AND req.transaction_id = sess.transaction_id
    LEFT JOIN sys.dm_exec_connections AS conn
        ON sess.session_id = conn.session_id
    OUTER APPLY sys.dm_exec_sql_text(req.sql_handle) AS txt
    OUTER APPLY sys.dm_exec_sql_text(conn.most_recent_sql_handle) AS recenttxt
WHERE atr.transaction_type != 2
      AND sess.session_id != @@spid
ORDER BY start_time ASC;

## Identificar problemas de desempenho de espera de concessão de memória

Se o seu tipo de espera superior for `RESOURCE_SEMAHPORE` e você não tiver um problema de uso de CPU alto, você pode ter um problema de concessão de memória em espera.

### [](https://docs.microsoft.com/pt-br/azure/azure-sql/database/monitoring-with-dmvs#determine-if-a-resource_semahpore-wait-is-a-top-wait)Determinar se um `RESOURCE_SEMAHPORE` espera for uma espera superior

Use a seguinte consulta para determinar se uma `RESOURCE_SEMAHPORE` espera é uma espera superior

SELECT wait\_type,

       SUM(wait\_time) AS total\_wait\_time\_ms

FROM sys.dm\_exec\_requests AS req

    JOIN sys.dm\_exec\_sessions AS sess

        ON req.session\_id = sess.session\_id

WHERE is\_user\_process = 1

GROUP BY wait\_type

ORDER BY SUM(wait\_time) DESC;

### Identificar instruções altas que consomem memória

Use a consulta a seguir para identificar instruções altas consumo de memória:

In [ ]:
SELECT wait_type,
       SUM(wait_time) AS total_wait_time_ms
FROM sys.dm_exec_requests AS req
    JOIN sys.dm_exec_sessions AS sess
        ON req.session_id = sess.session_id
WHERE is_user_process = 1
GROUP BY wait_type
ORDER BY SUM(wait_time) DESC;

### Identificar instruções altas que consomem memória

Use a consulta a seguir para identificar instruções altas consumo de memória:

In [ ]:
SELECT IDENTITY(INT, 1, 1) rowId,
    CAST(query_plan AS XML) query_plan,
    p.query_id
INTO #tmp
FROM sys.query_store_plan AS p
    JOIN sys.query_store_runtime_stats AS r
        ON p.plan_id = r.plan_id
    JOIN sys.query_store_runtime_stats_interval AS i
        ON r.runtime_stats_interval_id = i.runtime_stats_interval_id
WHERE start_time > '2018-10-11 14:00:00.0000000'
      AND end_time < '2018-10-17 20:00:00.0000000';
GO
;WITH cte
AS (SELECT query_id,
        query_plan,
        m.c.value('@SerialDesiredMemory', 'INT') AS SerialDesiredMemory
    FROM #tmp AS t
        CROSS APPLY t.query_plan.nodes('//*:MemoryGrantInfo[@SerialDesiredMemory[. > 0]]') AS m(c) )
SELECT TOP 50
    cte.query_id,
    t.query_sql_text,
    cte.query_plan,
    CAST(SerialDesiredMemory / 1024. AS DECIMAL(10, 2)) SerialDesiredMemory_MB
FROM cte
    JOIN sys.query_store_query AS q
        ON cte.query_id = q.query_id
    JOIN sys.query_store_query_text AS t
        ON q.query_text_id = t.query_text_id
ORDER BY SerialDesiredMemory DESC;

### Identificar as 10 principais concessão de memória

Use a consulta a seguir para identificar 10 principais concessão de memória:

In [ ]:
SELECT TOP 10
    CONVERT(VARCHAR(30), GETDATE(), 121) AS runtime,
       r.session_id,
       r.blocking_session_id,
       r.cpu_time,
       r.total_elapsed_time,
       r.reads,
       r.writes,
       r.logical_reads,
       r.row_count,
       wait_time,
       wait_type,
       r.command,
       OBJECT_NAME(txt.objectid, txt.dbid) 'Object_Name',
       LTRIM(RTRIM(REPLACE(
                              REPLACE(
                                         SUBSTRING(
                                                      SUBSTRING(
                                                                   text,
                                                                   (r.statement_start_offset / 2) + 1,
                                                                   ((CASE r.statement_end_offset
                                                                         WHEN -1 THEN
                                                                             DATALENGTH(text)
                                                                         ELSE
                                                                             r.statement_end_offset
                                                                     END - r.statement_start_offset
                                                                    ) / 2
                                                                   ) + 1
                                                               ),
                                                      1,
                                                      1000
                                                  ),
                                         CHAR(10),
                                         ' '
                                     ),
                              CHAR(13),
                              ' '
                          )
                  )
            ) stmt_text,
       mg.dop,                                               --Degree of parallelism
       mg.request_time,                                      --Date and time when this query requested the memory grant.
       mg.grant_time,                                        --NULL means memory has not been granted
       mg.requested_memory_kb / 1024.0 requested_memory_mb,  --Total requested amount of memory in megabytes
       mg.granted_memory_kb / 1024.0 AS granted_memory_mb,   --Total amount of memory actually granted in megabytes. NULL if not granted
       mg.required_memory_kb / 1024.0 AS required_memory_mb, --Minimum memory required to run this query in megabytes.
       max_used_memory_kb / 1024.0 AS max_used_memory_mb,
       mg.query_cost,                                        --Estimated query cost.
       mg.timeout_sec,                                       --Time-out in seconds before this query gives up the memory grant request.
       mg.resource_semaphore_id,                             --Non-unique ID of the resource semaphore on which this query is waiting.
       mg.wait_time_ms,                                      --Wait time in milliseconds. NULL if the memory is already granted.
       CASE mg.is_next_candidate --Is this process the next candidate for a memory grant
           WHEN 1 THEN
               'Yes'
           WHEN 0 THEN
               'No'
           ELSE
               'Memory has been granted'
       END AS 'Next Candidate for Memory Grant',
       qp.query_plan
FROM sys.dm_exec_requests AS r
    JOIN sys.dm_exec_query_memory_grants AS mg
        ON r.session_id = mg.session_id
           AND r.request_id = mg.request_id
    CROSS APPLY sys.dm_exec_sql_text(mg.sql_handle) AS txt
    CROSS APPLY sys.dm_exec_query_plan(r.plan_handle) AS qp
ORDER BY mg.granted_memory_kb DESC;

## Calcular os tamanhos de banco de dados e objetos

A seguinte consulta retorna o tamanho do seu banco de dados (em megabytes):

In [4]:
-- Calculates the size of the database.
SELECT SUM(CAST(FILEPROPERTY(name, 'SpaceUsed') AS bigint) * 8192.) / 1024 / 1024 AS DatabaseSizeInMB
FROM sys.database_files
WHERE type_desc = 'ROWS';
GO

(1 row affected)

Total execution time: 00:00:00.059

DatabaseSizeInMB
"15881,000000"


In [ ]:
A consulta a seguir retorna o tamanho do dos objetos individuais (em megabytes) no seu banco de dados:

In [3]:
-- Calculates the size of individual database objects.
SELECT sys.objects.name, SUM(reserved_page_count) * 8.0 / 1024
FROM sys.dm_db_partition_stats, sys.objects
WHERE sys.dm_db_partition_stats.object_id = sys.objects.object_id
GROUP BY sys.objects.name;
GO

(162 rows affected)

Total execution time: 00:00:00.106

name,(No column name)
Ajuda,"0,203125"
Area,"0,015625"
AreaInteresse,"22,945312"
AssociacaoCurso,"0,015625"
Auditoria,"8,976562"
Bairro,"5,414062"
Campanha,"0,000000"
CampoRelatorio,"3,593750"
CategoriaOportunidade,"0,015625"
Cidade,"1,093750"


## Monitoramento de conexões

É possível usar a exibição [sys.dm\_exec\_connections](https://docs.microsoft.com/pt-br/sql/relational-databases/system-dynamic-management-views/sys-dm-exec-connections-transact-sql) para recuperar informações sobre as conexões estabelecidas com um servidor específico e uma instância gerenciada e os detalhes de cada conexão. Além disso, a exibição [sys.dm\_exec\_sessions](https://docs.microsoft.com/pt-br/sql/relational-databases/system-dynamic-management-views/sys-dm-exec-sessions-transact-sql) é útil ao recuperar informações sobre todas as conexões de usuário e tarefas internas ativas.

A consulta a seguir recupera as informações sobre a conexão atual:

In [ ]:
SELECT
    c.session_id, c.net_transport, c.encrypt_option,
    c.auth_scheme, s.host_name, s.program_name,
    s.client_interface_name, s.login_name, s.nt_domain,
    s.nt_user_name, s.original_login_name, c.connect_time,
    s.login_time
FROM sys.dm_exec_connections AS c
JOIN sys.dm_exec_sessions AS s
    ON c.session_id = s.session_id
WHERE c.session_id = @@SPID;

<div style="background-color:#3b2e58;color:#e6e6e6">
 Observação
</div>
<div style="background-color:#3b2e58;color:#e6e6e6">
Ao executar as exibições sys.dm\_exec\_requests e sys.dm\_exec\_sessions, se você tiver a permissão EXIBIR ESTADO DO BANCO DE DADOS no banco de dados, verá todas as sessões em execução no banco de dados; caso contrário, verá apenas a sessão atual.

</div>

## Monitorar o uso de recursos

Você pode monitorar o uso de recursos do Banco de Dados SQL do Azure usando a [Análise de Desempenho de Consultas de Banco de Dados SQL](https://docs.microsoft.com/pt-br/azure/azure-sql/database/query-performance-insight-use). Para o Banco de Dados SQL do Azure e o Instância Gerenciada de SQL do Azure, você pode monitorar usando [Repositório de Dados de Consultas](https://docs.microsoft.com/pt-br/sql/relational-databases/performance/monitoring-performance-by-using-the-query-store).

Você também pode monitorar o uso com estes modos de exibições:

- Banco de Dados SQL do Azure: [sys.dm\_db\_resource\_stats](https://docs.microsoft.com/pt-br/sql/relational-databases/system-dynamic-management-views/sys-dm-db-resource-stats-azure-sql-database)
- Instância Gerenciada de SQL do Azure: [Sys.server\_resource\_stats](https://docs.microsoft.com/pt-br/sql/relational-databases/system-catalog-views/sys-server-resource-stats-azure-sql-database)
- Tanto o Banco de Dados SQL do Azure quanto a Instância Gerenciada de SQL do Azure: [sys.resource\_stats](https://docs.microsoft.com/pt-br/sql/relational-databases/system-catalog-views/sys-resource-stats-azure-sql-database)

### sys.dm\_db\_resource\_stats

Você pode usar a exibição [sys.dm\_db\_resource\_stats](https://docs.microsoft.com/pt-br/sql/relational-databases/system-dynamic-management-views/sys-dm-db-resource-stats-azure-sql-database) em cada banco de dados. A exibição sys.dm\_db\_resource\_stats mostra dados de uso de recursos recentes em relação à camada de serviço. A porcentagem média de CPU, E/S de dados, gravações de log e memória é registrada a cada 15 segundos e armazenada por 1 hora.

Como essa exibição oferece uma visão mais granular do uso de recursos, use sys.dm\_db\_resource\_stats primeiro para qualquer análise de estado atual ou para solução de problemas. Por exemplo, esta consulta mostra o uso médio e máximo dos recursos do banco de dados atual durante a última hora:

In [ ]:
SELECT  
    AVG(avg_cpu_percent) AS 'Average CPU use in percent',
    MAX(avg_cpu_percent) AS 'Maximum CPU use in percent',
    AVG(avg_data_io_percent) AS 'Average data IO in percent',
    MAX(avg_data_io_percent) AS 'Maximum data IO in percent',
    AVG(avg_log_write_percent) AS 'Average log write use in percent',
    MAX(avg_log_write_percent) AS 'Maximum log write use in percent',
    AVG(avg_memory_usage_percent) AS 'Average memory use in percent',
    MAX(avg_memory_usage_percent) AS 'Maximum memory use in percent'
FROM sys.dm_db_resource_stats;


<span style="color: rgb(230, 230, 230); font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(23, 23, 23);">Para outras consultas, consulte os exemplos em&nbsp;</span> [sys.dm\_db\_resource\_stats](https://docs.microsoft.com/pt-br/sql/relational-databases/system-dynamic-management-views/sys-dm-db-resource-stats-azure-sql-database)<span style="color: rgb(230, 230, 230); font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(23, 23, 23);">.</span>

### sys.server\_resource\_stats

Você pode usar [sys.server\_resource\_stats](https://docs.microsoft.com/pt-br/sql/relational-databases/system-catalog-views/sys-server-resource-stats-azure-sql-database)retornar dados de uso de CPU, e/s e armazenamento para uma Instância Gerenciada de SQL do Azure. Os dados são coletados e agregados em intervalos de cinco minutos. Há uma linha para cada relatório de 15 segundos. Os dados retornados incluem uso de CPU, tamanho de armazenamento, utilização de e/s e SKU de instância gerenciada. Os dados históricos são retidos por aproximadamente 14 dias.

In [9]:
DECLARE @s datetime;  
DECLARE @e datetime;  
SET @s= DateAdd(d,-7,GetUTCDate());  
SET @e= GETUTCDATE();  
SELECT resource_name, AVG(avg_cpu_percent) AS Average_Compute_Utilization
FROM sys.server_resource_stats
WHERE start_time BETWEEN @s AND @e  
GROUP BY resource_name  
HAVING AVG(avg_cpu_percent) >= 80;

: Msg 208, Level 16, State 1, Line 5
Invalid object name 'sys.server_resource_stats'.

Total execution time: 00:00:00.003

### sys. resource\_stats

A exibição [sys.resource\_stats](https://docs.microsoft.com/pt-br/sql/relational-databases/system-catalog-views/sys-resource-stats-azure-sql-database) no banco de dados mestre tem mais informações que podem ajudá-lo a monitorar o desempenho do banco de dados em sua camada de serviço e tamanho da computação específicos. Os dados são coletados a cada 5 minutos e são mantidos por aproximadamente 14 dias. Essa exibição é útil para uma análise de histórico de longo prazo de como seu banco de dados usa recursos.

O grafo a seguir mostra o uso de recursos da CPU para um banco de dados Premium com o tamanho da computação P2 para cada hora em uma semana. Esse grafo começa em uma segunda-feira, mostra cinco dias úteis e, depois, um fim de semana, em que ocorrem muito menos atividades no aplicativo.

![Uso de recursos de banco de dados](https://docs.microsoft.com/pt-br/azure/azure-sql/database/media/monitoring-with-dmvs/sql_db_resource_utilization.png)

Segundo os dados, atualmente esse banco de dados tem um pico de carga de CPU de pouco mais de 50% de uso da CPU em relação ao tamanho da computação P2 (meio-dia de terça-feira). Se a CPU for o fator dominante no perfil de recursos do aplicativo, talvez você decida que P2 é o tamanho da computação certo para garantir que a carga de trabalho sempre seja adequada. Se você espera que um aplicativo cresça com o passar do tempo, é uma boa ideia ter um buffer de recursos adicional para que o aplicativo nunca atinja o limite do nível de desempenho. Aumentando o tamanho da computação, você pode ajudar a evitar erros visíveis para os clientes, que podem ocorrer quando um banco de dados não tem capacidade suficiente para processar solicitações de modo eficaz, especialmente em ambientes sensíveis à latência. Um exemplo é um banco de dados que dá suporte a um aplicativo que pinta páginas da Web com base nos resultados de chamadas de banco de dados.

Outros tipos de aplicativos podem interpretar o mesmo grafo de forma diferente. Por exemplo, se um aplicativo tentar processar dados de folha de pagamento por dia e incluir o mesmo gráfico, esse tipo de modelo de "trabalho em lotes" poderá funcionar bem em um tamanho da computação P1. O tamanho da computação P1 tem 100 DTUs, comparadas a 200 DTUs do tamanho da computação P2. O tamanho da computação P1 fornece que metade do desempenho do P2. Portanto, 50% de uso da CPU em P2 equivale a 100% de uso da CPU em P1. Se o aplicativo não tiver tempo limite, talvez não faça diferença se um trabalho demora 2 horas ou 2,5 horas para ser concluído, desde que seja concluído hoje. Um aplicativo dessa categoria provavelmente pode usar um tamanho da computação P1. Você pode tirar proveito do fato de que há períodos do dia em que o uso de recursos é menor, o que significa que um "pico grande" pode ser extrapolado para um dos ciclos mais tarde. O tamanho da computação P1 pode ser bom para esse tipo de aplicativo (e economiza dinheiro), desde que os trabalhos possam ser concluídos no horário todos os dias.

O mecanismo de banco de dados expõe informações de recurso consumido para cada banco de dados ativo na exibição sys.resource\_stats do banco de dados mestre em cada servidor. Os dados na tabela são agregados em intervalos de cinco minutos. Com as camadas de serviço Básica, Standard e Premium, os dados podem demorar mais de 5 minutos para aparecer na tabela, o que significa que esses dados são mais úteis para análise histórica do que para análise em tempo real. Consulte a exibição sys.resource\_stats para ver o histórico recente de um banco de dados e validar se a reserva escolhida ofereceu o desempenho desejado quando necessário.

  
<div style="background-color:#3b2e58;color:#e6e6e6">
 Observação
</div>
<div style="background-color:#3b2e58;color:#e6e6e6">
No Banco de Dados SQL do Azure, você precisa estar conectado ao banco de dados mestre para consultar sys.resource\_stats nos exemplos a seguir.
</div>
Este exemplo mostra como os dados nesse modo de exibição são expostos:

In [3]:
SELECT TOP 10 *
FROM sys.resource_stats
WHERE database_name = 'resource1'
ORDER BY start_time DESC;

: Msg 208, Level 16, State 1, Line 1
Invalid object name 'sys.resource_stats'.

Total execution time: 00:00:00.002

O exemplo seguinte mostra diferentes maneiras de usar o modo de exibição de catálogo sys.resource\_stats para obter informações sobre como o banco de dados usa recursos:

1. Para examinar o uso de recursos da última semana para o banco de dados userdb1, você pode executar esta consulta:

In [ ]:
SELECT *
FROM sys.resource_stats
WHERE database_name = 'userdb1' AND
    start_time > DATEADD(day, -7, GETDATE())
ORDER BY start_time DESC;

<span style="; font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; ">2.Para avaliar como sua carga de trabalho se ajusta ao tamanho da computação, você precisa fazer uma pesquisa detalhada de cada aspecto das métricas de recursos: CPU, leituras, gravações, número de trabalhadores e número de sessões. Veja uma consulta revisada usando&nbsp;</span> <span style="box-sizing: inherit; font-weight: 600; outline-color: inherit;  font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; ">sys.resource_stats</span> <span style=" font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; ">&nbsp;para relatar os valores médio e máximo dessas métricas de recursos:</span>

In [ ]:
SELECT
    avg(avg_cpu_percent) AS 'Average CPU use in percent',
    max(avg_cpu_percent) AS 'Maximum CPU use in percent',
    avg(avg_data_io_percent) AS 'Average physical data IO use in percent',
    max(avg_data_io_percent) AS 'Maximum physical data IO use in percent',
    avg(avg_log_write_percent) AS 'Average log write use in percent',
    max(avg_log_write_percent) AS 'Maximum log write use in percent',
    avg(max_session_percent) AS 'Average % of sessions',
    max(max_session_percent) AS 'Maximum % of sessions',
    avg(max_worker_percent) AS 'Average % of workers',
    max(max_worker_percent) AS 'Maximum % of workers'
FROM sys.resource_stats
WHERE database_name = 'userdb1' AND start_time > DATEADD(day, -7, GETDATE());

3.Com essas informações sobre os valores médio e máximo de cada métrica de recurso, você pode avaliar como sua carga de trabalho se ajusta ao tamanho da computação escolhido. Normalmente, os valores médios de sys.resource\_stats oferecem uma boa linha de base a ser usada em comparação ao tamanho de destino. Deve ser seu cartão de medida principal. Por exemplo, você pode estar usando a camada de serviço Standard com o tamanho da computação S2. Os percentuais médios de uso da CPU e leituras e gravações de E/S estão abaixo de 40%, o número médio de trabalhadores está abaixo de 50 e o número médio de sessões é inferior a 200. Talvez a carga de trabalho se enquadre no tamanho da computação S1. É fácil ver se o banco de dados se encaixa nos limites de sessão e de trabalho. Para ver se um banco de dados se enquadra em um tamanho da computação inferior em relação à CPU, leituras e gravações, divida o número de DTU do tamanho da computação inferior pelo número de DTU do tamanho da computação atual e multiplique o resultado por 100:

```
S1 DTU / S2 DTU * 100 = 20 / 50 * 100 = 40
```

O resultado é a diferença de desempenho relativa entre os dois tamanhos da computação em percentual. Se seu uso de recursos não ultrapassar esse valor, sua carga de trabalho poderá se enquadrar no tamanho da computação inferior. No entanto, você precisa examinar todos os intervalos de valores de uso de recursos e determinar, pelo percentual, com que frequência a carga de trabalho do banco de dados se enquadraria no tamanho da computação inferior. A consulta a seguir produz o percentual de ajuste por dimensão de recurso, com base no limite de 40% que calculamos neste exemplo:

In [ ]:
SELECT
     (COUNT(database_name) - SUM(CASE WHEN avg_cpu_percent >= 40 THEN 1 ELSE 0 END) * 1.0) / COUNT(database_name) AS 'CPU Fit Percent',
     (COUNT(database_name) - SUM(CASE WHEN avg_log_write_percent >= 40 THEN 1 ELSE 0 END) * 1.0) / COUNT(database_name) AS 'Log Write Fit Percent',
     (COUNT(database_name) - SUM(CASE WHEN avg_data_io_percent >= 40 THEN 1 ELSE 0 END) * 1.0) / COUNT(database_name) AS 'Physical Data IO Fit Percent'
 FROM sys.resource_stats
 WHERE database_name = 'userdb1' AND start_time > DATEADD(day, -7, GETDATE());

Com base na camada de serviço do banco de dados, você pode decidir se sua carga de trabalho se enquadra no tamanho da computação inferior. Se a meta de carga de trabalho do banco de dados é de 99,9% e a consulta anterior retorna valores superiores a 99,9% para as três dimensões de recursos, é provável que sua carga de trabalho se enquadre no tamanho da computação inferior.

A avaliação do percentual de ajuste também traz informações que indicam se você deve ir ao próximo tamanho da computação para chegar à sua meta. Por exemplo, userdb1 mostra o seguinte uso da CPU para a semana passada:

<caption class="visually-hidden" style="box-sizing: inherit; outline-color: inherit; border-width: 0px 0px 1px; border-style: solid; border-color: var(--theme-border); border-image: initial; clip: rect(1px, 1px, 1px, 1px); clip-path: inset(50%); height: 1px; margin: -1px; overflow: hidden; padding: 1.125rem; position: absolute; width: 1px; overflow-wrap: normal; font-size: 0.8rem; font-weight: 600; letter-spacing: 2px; text-transform: uppercase; text-align: right;">TABELA 1</caption>
				
| Percentual médio da CPU | Percentual máximo da CPU |
| --- | --- |
| 24,5 | 100,00 |

A média da CPU é de aproximadamente um quarto do limite do tamanho da computação, o que se ajustaria bem ao tamanho da computação do banco de dados. No entanto, o valor máximo mostra que o banco de dados atinge o limite do tamanho da computação. Você precisa ir para o próximo tamanho da computação? Avalie quantas vezes sua carga de trabalho atinge 100% e compará-la à meta de carga de trabalho de seu banco de dados.

In [ ]:
SELECT
    (COUNT(database_name) - SUM(CASE WHEN avg_cpu_percent >= 100 THEN 1 ELSE 0 END) * 1.0) / COUNT(database_name) AS 'CPU fit percent'
    ,(COUNT(database_name) - SUM(CASE WHEN avg_log_write_percent >= 100 THEN 1 ELSE 0 END) * 1.0) / COUNT(database_name) AS 'Log write fit percent'
    ,(COUNT(database_name) - SUM(CASE WHEN avg_data_io_percent >= 100 THEN 1 ELSE 0 END) * 1.0) / COUNT(database_name) AS 'Physical data IO fit percent'
    FROM sys.resource_stats
    WHERE database_name = 'userdb1' AND start_time > DATEADD(day, -7, GETDATE());

1. Se essa consulta retornar um valor inferior a 99,9% para qualquer uma das três dimensões de recursos, considere passar para o tamanho da computação superior ou use técnicas de ajuste de aplicativo para reduzir a carga no banco de dados.
    
2. Este exercício também leva em consideração o aumento projetado da carga de trabalho no futuro.
    

Para pools elásticos, você pode monitorar bancos de dados autônomos no pool com as técnicas descritas nesta seção. Mas você também pode monitorar o pool como um todo. Para saber mais, veja [Monitorar e gerenciar um pool elásticos](https://docs.microsoft.com/pt-br/azure/azure-sql/database/elastic-pool-overview).